In [1]:
import logging
from pathlib import Path
import os
import sys
import mne
import torch
import numpy as np
import bm
from bm import play
from bm.train import main
from bm.events import Word
from matplotlib import pyplot as plt
from IPython import display as disp

mne.set_log_level(False)
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
os.chdir(main.dora.dir.parent)
os.environ['NO_DOWNLOAD'] = '1'
from tqdm import tqdm
import IPython.display as ipd
from torch.nn.functional import conv2d, conv1d
import torch.nn.functional as F
from bm.losses import ClipLoss

Hostname bagration not defined in /conf/study_paths/study_paths.yaml. Using default paths.


In [2]:
sig='342eaad6'

In [3]:
solver = play.get_solver_from_sig(sig) 

INFO:bm.play:Loading solver from XP 342eaad6. Overrides used: ['model=clip_conv', 'dset.selections=["gwilliams2022"]', 'dset.bandpass=true', 'dset.bandpass_high=0.1', 'dset.bandpass_lower=40.0', 'seed=2038', 'dset.tmin=-0.75', 'dset.tmax=3.25']
{'wandb': {'use_wandb': True, 'project': 'brainmagick', 'group': 'brainmagick-group'}, 'num_prints': 5, 'device': 'cuda:1', 'num_workers': 5, 'verbose': 0, 'show': 0, 'download_only': False, 'slurm': {'mem_per_gpu': 100, 'time': 4320}, 'continue_sig': None, 'continue_best': True, 'seed': 2038, 'dummy': None, 'cache': '/home/zubrikhina/brainmagick_MICCAI/cache', 'features_models': './features_models', 'early_stop_patience': 10, 'eval_every': 1, 'eval_train_set': False, 'optim': {'name': 'adam', 'shuffle': False, 'lr': 0.0003, 'beta2': 0.999, 'epochs': 200, 'batch_size': 256, 'loss': 'clip', 'weight_decay': 0.0, 'use_weighting': False, 'max_batches': 1200, 'svd': 0.0, 'negatives': None, 'negative_pool_size': None}, 'clip': {'linear': None, 'twin':

/home/zubrikhina/anaconda3/envs/bm/lib/python3.8/site-packages/flashy/loggers/tensorboard.py:47: UserWarning: tensorboard package was not found: use pip install tensorboard
  warnings.warn("tensorboard package was not found: use pip install tensorboard")
wandb: Currently logged in as: almachan2358 (gnn-neuro). Use `wandb login --relogin` to force relogin


In [4]:
clip =ClipLoss(probabilities=True)

In [5]:
def _get_one_segments_and_vocabs(solver):
    from scripts.run_eval_probs import _get_extra_info
    split ='train'
    segments,vocab, estimates,outputs,features_masks, reject_masks = [], [],[], [],[], []
    dset = getattr(solver.datasets, split)
    loader = solver.make_loader(dset, shuffle=False)
    test_features = solver.datasets.test.datasets[0].features
    for idx, batch in tqdm(enumerate(loader)):
        #features = test_features.extract_features(batch.features, solver.used_features.keys())
        with torch.no_grad():
            estimate, output, features_mask, reject_mask = solver._process_batch(batch)
            data,  words, word_segs= _get_extra_info(batch, solver.args.dset.sample_rate)
        if idx>1:
            break
    return batch,features_mask, estimate, output, word_segs,vocab

In [6]:
batch,features_mask, estimate, output, word_segs,vocab = _get_one_segments_and_vocabs(solver)

2it [00:11,  5.65s/it]


In [7]:
clip.forward(estimate, output,features_mask)

soft-clip


tensor(0.6602, device='cuda:1')

In [8]:
clip

ClipLoss()